The framework is a fork from FLEPSeq. Code can be found here https://github.com/WeberJoachim/Viz_bdg_and_nanopore_bam
Make sure to use a unix machine, since pysam and windows is pain.
Dont worry this takes forever and is not at all optimised. But hey it works.

In [ ]:
# the representative gene_id is available at:
# https://www.arabidopsis.org/download_files/Genes/Araport11_genome_release/Araport11_blastsets/Araport11_seq_20220914_representative_gene_model.gz

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mp
import pandas as pd
import numpy as np
import pysam


%config InlineBackend.figure_format = 'retina'

araport11_isoform_path = './FLEPSeq/genome_lib/araport11.representative.gene_model.bed'
araport11_isoform = pd.read_csv(araport11_isoform_path, sep='\t', 
                                names=['chrom', 'chromStart', 'chromEnd', 'name', 
                                       'score', 'strand', 'thickStart', 'thickEnd', 
                                       'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
                               )
araport11_isoform['gene_id'] = araport11_isoform['name'].map(lambda x: x.split('.')[0])
araport11_isoform.set_index('gene_id', inplace=True)


In [ ]:
class IGV(object):
    '''IGV(gene_id)
    
    A class for bam alignment visualization, base on the bam file and the annotation bed12 file

    Attributes:
        gene_id: A string type of gene_id list in bed12 file.
    '''

    def __init__(self, gene_id):
        self.gene_id = gene_id
        self._get_gene_info()
        self.max = self.end
        self.min = self.start
        self.bam_list = {}
        self.bedgraph_list = []
        self.labels_bam = []
        self.labels_bedgraph = []

    
    def _get_gene_info(self):
        '''
        Get gene info from bed file
        '''
        gene_info = araport11_isoform.loc[self.gene_id]  # araport11_isoform loaded from pandas
        self.chrom = gene_info.chrom
        self.start = gene_info.chromStart
        self.end = gene_info.chromEnd
        self.strand = gene_info.strand
        self.thickStart = gene_info.thickStart
        self.thickEnd = gene_info.thickEnd
        self.blockCount = gene_info.blockCount
        self.strand_boo = False if self.strand == '+' else True
        self.blockSizes = np.fromstring(gene_info.blockSizes, sep=',', dtype='int')
        self.blockStarts = np.fromstring(gene_info.blockStarts, sep=',', dtype='int') + self.start # 0 base

    def _plot_gene_model(self, ax, gene_color='k'):
        # plot TSS
        small_relative = 0.05 * (self.max-self.min) 
        arrowprops = dict(arrowstyle="-|>", connectionstyle="angle", color=gene_color)
        if self.strand == '+':
            ax.annotate('', xy=(self.start+small_relative, .6), xytext=(self.start, 0), arrowprops=arrowprops)
        else:
            ax.annotate('', xy=(self.end-small_relative, .6), xytext=(self.end, 0), arrowprops=arrowprops)
        ax.plot([self.start, self.end], [0, 0], color=gene_color) 

        height = .3 # the height of gene model

        for exonstart, size in zip(self.blockStarts, self.blockSizes):
            if exonstart == self.start and exonstart+size == self.end:
                utr_size = self.thickStart-self.start
                utr = mp.Rectangle((exonstart, 0-height/2), utr_size, height, color=gene_color, linewidth=0)
                ax.add_patch(utr)
                utr_size = self.end-self.thickEnd
                utr = mp.Rectangle((self.thickEnd, 0-height/2), utr_size, height, color=gene_color, linewidth=0)
                ax.add_patch(utr)
                exon = mp.Rectangle((self.thickStart, 0-height), self.thickEnd-self.thickStart, height*2, color=gene_color, linewidth=0)
                ax.add_patch(exon)
            elif exonstart + size <= self.thickStart:
                # only 5'/ 3'UTR
                utr = mp.Rectangle((exonstart, 0-height/2), size, height, color=gene_color, linewidth=0)
                ax.add_patch(utr)
            elif exonstart < self.thickStart and exonstart + size > self.thickStart:
                # exon with 5' / 3' UTR 
                utr_size = self.thickStart-exonstart
                utr = mp.Rectangle((exonstart, 0-height/2), utr_size, height, color=gene_color, linewidth=0)
                exon = mp.Rectangle((exonstart+utr_size, 0-height), size-utr_size, height*2, color=gene_color, linewidth=0)
                ax.add_patch(utr)
                ax.add_patch(exon)
            elif exonstart >= self.thickStart and exonstart + size <= self.thickEnd:
                # regular exon
                exon = mp.Rectangle((exonstart, 0-height), size, height*2, color=gene_color, linewidth=0)
                ax.add_patch(exon)
            elif exonstart < self.thickEnd and exonstart + size > self.thickEnd:
                # exon with 3' / 5' UTR
                utr_size = exonstart + size - self.thickEnd
                utr = mp.Rectangle((self.thickEnd, 0-height/2), utr_size, height, color=gene_color, linewidth=0)
                exon = mp.Rectangle((exonstart, 0-height), size-utr_size, height*2, color=gene_color, linewidth=0)
                ax.add_patch(utr)
                ax.add_patch(exon)
            elif exonstart >= self.thickEnd:
                # only 3'/ 5'UTR
                utr = mp.Rectangle((exonstart, 0-height/2), size, height, color=gene_color, linewidth=0)
                ax.add_patch(utr)
            
        ax.annotate(self.gene_id, xy=((self.start+self.end)/2, 0.8), ha='center')
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.yaxis.set_major_locator(ticker.NullLocator())
        ax.xaxis.set_major_locator(ticker.NullLocator())
        ax.xaxis.set_ticks_position('none')
        ax.set_ylim(-.5, 1)
        
        
    def _find_exon(self, read):
        BAM_CREF_SKIP = 3 #BAM_CREF_SKIP
        blockStart = []
        blockSize = []
        match_or_deletion = {0, 2, 7, 8} # only M/=/X (0/7/8) and D (2) are related to genome position
        exon_start = read.reference_start
        length = 0
        for op, nt in read.cigartuples:
            if op in match_or_deletion:
                length += nt
            elif op == BAM_CREF_SKIP:
                blockStart.append(exon_start)
                blockSize.append(length)
                exon_start += length+nt
                length = 0
        blockStart.append(exon_start)
        blockSize.append(length)
        return zip(blockStart, blockSize)
        
    def _plot_bam(self, ax, read_list, bam_num, read_color='#5D93C4'):
        ypos = 0
        height = .6
        for read in read_list:
            line = mp.Rectangle((read.reference_start, ypos-height/4), read.reference_length, height/2, color='#A6A6A6', linewidth=0)
            ax.add_patch(line)
            for block_start, block_size in self._find_exon(read):
                exon = mp.Rectangle((block_start, ypos-height), block_size, height*2, color=read_color, linewidth=0)
                ax.add_patch(exon)
            ypos += -1

        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.set_major_locator(ticker.NullLocator())
        ax.set_ylim(ypos-int(len(read_list)*.1), 1+int(len(read_list)*.25))
   

        if bam_num != len(self.bam_list) + len(self.bedgraph_list):
            ax.xaxis.set_major_locator(ticker.NullLocator())
            ax.xaxis.set_ticks_position('none')
        else:
            ax.set_xlabel('Length (nt)')

         


    def _filter_reads(self, read, five_prime_threshold=150):
        # discard the antisense read
        if read.is_reverse is not self.strand_boo:
            return True
        # discard reads generated from upstream gene
        if read.is_reverse and read.reference_end-self.end > five_prime_threshold:
            return True
        elif not read.is_reverse and self.start - read.reference_start > five_prime_threshold:
            return True

        return False

    def _sort_bam(self, read_list, sort_method):
        if sort_method == '3':
            if self.strand == '+':
                read_list.sort(key=lambda read: read.reference_end)
            else:
                read_list.sort(key=lambda read: read.reference_start, reverse=True)
        elif sort_method == '5':
            if self.strand == '+':
                read_list.sort(key=lambda read: read.reference_start)
            else:
                read_list.sort(key=lambda read: read.reference_end, reverse=True)
        
          

    def add_bam(self, *bam_paths, sort_method='3', label = ""):
        '''add_bam(self, *bam_paths, read_type={'polya', 'elongating'}, sort_method='3',)

        Add read object to the self.bam_list

        Args:
            *bam_paths: An the bam file path. default: {'polya', 'elongating'}.
            
            read_type: A set contained any of {'polya', 'elongating', 'elongating_5lost', 
                'polya_5lost', 'splicing_intermediate', 'elongating_3_mapping_low_accuracy', 
                'polya_3_not_in_last_exon'}
                Determines what kinds of reads to plot.
                
            sort_method: {'3', '5', 'ir'}, default '3'
                Sort reads strategy. -'3': sort by 3' end position of reads.
                -'5': sort by 5' end position of reads. 'ir': sort by retained
                intron.
        '''
        self.labels_bam.append(label)

        for bam_path in bam_paths:
           
            with pysam.AlignmentFile(bam_path, 'rb') as inbam:
                non_polya_read = []
                
                # extend search range downstream 1000nt
                if self.strand == '-':
                    start = self.start-1000 if self.start > 1000 else 1
                    end = self.end
                else:
                    end = self.end+1000 if self.end+1000 < inbam.get_reference_length(self.chrom) else inbam.get_reference_length(self.chrom)
                    start = self.start
                    
                for read in inbam.fetch(self.chrom, start, end):
                    # filter read
                    if self._filter_reads(read):
                        continue
                    
                   
                    
                    else:
                        non_polya_read.append(read)

            # sort reads
            self._sort_bam(non_polya_read, sort_method)
            # merge
            self.bam_list[bam_path] = non_polya_read
    

    def add_bedgraph(self, bedgraph_path):
        complete_bedgraph = pd.read_csv(bedgraph_path, sep='\t', header=None, names=['chrom', 'start', 'end', 'value'], low_memory=False)
        self.bedgraph_list.append(complete_bedgraph)
               
                 


    def _plot_bedgraph(self, ax, bedgraph,  extend_3_prime_by, bedgraph_color = "darkgrey"):

               

        if self.strand == '-':
            subset_bedgraph = bedgraph[(bedgraph['chrom'] == self.chrom) & (bedgraph['start'] >= self.start - extend_3_prime_by) & (bedgraph['end'] <= self.end)]
            subset_bedgraph = subset_bedgraph.iloc[::-1]
     
        else:
            subset_bedgraph = bedgraph[(bedgraph['chrom'] == self.chrom) & (bedgraph['start'] >= self.start) & (bedgraph['end'] <= self.end + extend_3_prime_by)]


        
      
        values = subset_bedgraph['value']
        max_value = max(values)
        ymax = 1.1
         
      
        for index, line in subset_bedgraph.iterrows():
            start = line["start"]
            end = line["end"]
            value = line["value"]

            width = end - start + 2
            height = value/max_value
            
            
            rectangle = mp.Rectangle((start, 0), width,  height , color=bedgraph_color, linewidth=0)
            

            ax.add_patch(rectangle)
           
      

        
        ax.set_ylim(0, ymax)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
            

        


        #add xaxis to each plot


    def plot(self, height=3, height_bedgraph = 3, width=6, extend_3_prime_by = 0):
        '''plot(self, height=3, width=6)

        Plot the alignment

        Args:
            height: the height of the figure, default 3.
            width: the width of the figure, default 6.
        '''
        nrows = len(self.bam_list) + len(self.bedgraph_list) +1
       
        height_ratios = []

        

        
        
        for i in self.bedgraph_list:
                height_ratios.append(height_bedgraph)

        height_ratios.append(0.6)
        
        for bam_path in self.bam_list:
                height_ratios.append(height)
      

        fig, ax = plt.subplots(
            nrows=nrows, 
            gridspec_kw={'height_ratios': height_ratios},
            figsize=(width, sum(height_ratios)),
            sharex = True,
        )


        for i, sub_graph in enumerate(self.bedgraph_list):
    
            self._plot_bedgraph(ax[i], sub_graph, extend_3_prime_by)
            



        # plot gene_model
        if len(self.bam_list) == 0 and len(self.bedgraph_list) == 0:
            self._plot_gene_model(ax)
        else:
            self._plot_gene_model(ax[len(self.bedgraph_list)])



        ## plot bedgraph files
      
        ### plot bam files
        for i, bam_path in enumerate(self.bam_list, 1):
            ax[i].set_title(self.labels_bam[i-1], y = 0.95, pad = 3, fontsize='medium', loc = "left")
            i = i + len(self.bedgraph_list)
            self._plot_bam(ax[i], self.bam_list[bam_path], i)
            
        

        ax_ = ax[i]
        step = (self.max-self.min)//400*100  # ticklabels of x_axis


        
        if self.strand == '+':
            #xticks = np.arange(self.start, self.end+step, step)
            xticks = np.arange(self.min, self.max+step, step)
            ax_.set_xticks(xticks)
            ax_.set_xticklabels(xticks-self.min)
            ax_.set_xlim(self.min, self.max + extend_3_prime_by)
        else:
            #xticks = np.arange(self.end, self.start-step, -step)
            xticks = np.arange(self.max, self.min-step, -step)
            ax_.set_xticks(xticks)
            ax_.set_xticklabels(self.max-xticks) 
            ax_.set_xlim(self.min - extend_3_prime_by, self.max)
            ax_.invert_xaxis()

           
        plt.subplots_adjust(hspace=0.2)
        plt.savefig(f'{self.gene_id}.igv.svg', format='svg', bbox_inches='tight')


Here i used the combined alignments (nanopore bams), the bedgraph from chipseq, the bedgraph from 3'prime endseq and the bedgraph from RNAseq.

In [ ]:
%time

#WT

bedgraph1 = 'rnaseq_bgs/U1_70K_rep2.bg'
bedgraph2 = 'rnaseq_bgs/U1_C_rep2.bg'
bedgraph3 = 'rnaseq_bgs/WT_rep2.bg'
bedgraph4 = '3prime_bgs/U1_70K_R2_trimmed_un_sorted.bg'
bedgraph5 = '3prime_bgs/U1_C_R2_trimmed_un_sorted.bg'
bedgraph6 = '3prime_bgs/WT_R2_trimmed_un_sorted.bg'
bedgraph7 = 'pol_bigwig/70K_Pol_FE_mean.bedgraph'
bedgraph8 = 'pol_bigwig/U1C_Pol_FE_mean_2u3.bedgraph'
bedgraph9 = 'pol_bigwig/WT_Pol_FE_mean.bedgraph'


nanopore_bam_file1 = '20230201_PAG71173_RNA_Thaliana_RNA002_NA_Laubinger_Laubinger_sorted_merged.bam'
nanopore_bam_file2 = '20230202_PAG72817_RNA_Thaliana_RNA002_NA_Laubinger2_Laubinger_sorted_merged.bam'
nanopore_bam_file3 = '20230201_PAG73257_RNA_Thaliana_RNA002_NA_Laubinger_Laubinger_sorted_merged.bam'


gene_ids = ["AT1G78150","AT3G54900","AT3G56050","AT5G04550","AT3G02832","AT1G61990","AT5G65720","AT2G26140","AT1G05870","AT4G25080","AT4G34180","AT1G70185","AT3G01500","AT1G30450","AT1G64770","AT2G42380","AT3G53130","AT4G01050","AT4G14160","AT5G61310","AT5G21070","AT1G01510","AT1G07320","AT1G19000","AT1G32200","AT1G15405","AT3G51260","AT3G52420","AT5G58370","AT1G09130","AT3G45050","AT1G27435","AT4G16950","AT1G02720","AT5G17310","AT2G36300","AT1G57540","AT1G33490","AT3G56800","AT2G30550","AT1G18745","AT1G72320","AT1G63855", "AT5G65840","AT3G54440","AT3G50560","AT1G29465","AT1G64990","AT4G34215","AT1G30270","AT2G35490","AT4G05400","AT5G01775","AT4G32570","AT5G45260","AT3G10270","AT4G25990","AT5G12170","AT1G26120","AT1G22430","AT1G07650","AT3G62750","AT4G39235","AT2G24150","AT5G19460","AT3G18790","AT2G47760","AT2G46710","AT2G04630","AT2G30420","AT3G02070","AT3G24320","AT3G58110","AT3G15820","AT2G30740","AT5G66360","AT3G16260","AT5G07240","AT1G78260","AT4G15630","AT1G54730", "AT3G24518","AT1G01020","AT3G55850","AT5G06165","AT1G08230","AT4G34480","AT5G46270","AT4G11830","AT3G56210","AT5G43100","AT2G39850","AT1G47570","AT1G12750","AT1G72030","AT5G02645","AT5G26667","AT3G14840", "AT5G61220","AT4G19110","AT1G60260","AT1G69550","AT2G31440", "AT2G18300","AT5G15020","AT3G63190","AT4G10320","AT1G23170","AT1G68830","AT1G71720","AT3G21250","AT5G10690", "AT2G24270","AT4G00150","AT2G01140","AT2G26250","AT2G25840","AT1G04350","AT3G20230","AT2G28950","AT2G45690","AT5G67330","AT5G57270","AT3G53350","AT5G15410","AT1G17840","AT5G36290","AT5G65310","AT3G23080","AT4G19640","AT5G59850","AT1G21980","AT2G18160","AT1G64385","AT4G29360","AT3G27340","AT5G48930","AT3G51140","AT1G25420","AT5G63140","AT5G49760","AT2G39681","AT4G36800","AT3G07100","AT2G36830","AT1G66500","AT2G28780","AT1G72910","AT1G56330","AT3G60750","AT1G21590","AT3G21060","AT3G55010","AT1G43620","AT5G12040","AT5G54810","AT5G08640","AT2G18280","AT1G54780","AT5G59920","AT1G74070","AT5G48150","AT5G11420","AT1G49840","AT1G26920","AT4G35460","AT2G21790","AT1G03600","AT5G62140","AT4G28860","AT3G14330","AT5G53450","AT4G00238","AT1G69580","AT5G61440","AT1G04410","AT4G14620","AT5G52780","AT1G78020","AT4G39400","AT3G28460","AT4G37300","AT2G36060","AT1G09660","AT1G55370","AT1G44100","AT4G29080","AT3G17000","AT3G58990","AT3G21220","AT2G34810","AT4G32010","AT2G46400","AT3G02730","AT5G13930","AT3G52060","AT1G24530","AT5G04240","AT3G06150","AT1G12110","AT2G32070","AT4G26080","AT1G24706","AT4G02100","AT4G25030","AT3G10050","AT5G58600","AT5G17230","AT3G23170","AT3G59068","AT1G64780","AT5G03290","AT4G31170","AT3G53460","AT2G20670","AT3G49670","AT3G10610","AT4G17090","AT3G11410","AT1G49500","AT2G01490","AT5G28500","AT1G25450","AT5G56350","AT3G09250","AT3G24440","AT3G18490","AT5G06150","AT3G59630","AT3G14310","AT2G16090","AT1G56300","AT2G15280","AT4G04570","AT5G46450","AT2G16710","AT4G12390","AT4G14040","AT5G41400","AT1G77480","AT2G27310","AT1G78820","AT5G02530","AT5G27850","AT5G25460","AT5G63190","AT3G18035","AT1G17550","AT2G37240","AT2G35860","AT3G52930","AT4G37870","AT5G27950","AT3G18890","AT5G41410","AT2G14660","AT3G06430","AT3G06120","AT1G75560","AT5G53420","AT5G13650","AT1G49410","AT5G55620","AT3G18780","AT4G37330","AT3G01390","AT3G26810","AT2G36880","AT2G45560","AT5G56530","AT3G56010","AT4G01037","AT1G08810","AT1G76670"]


# overlap repressed composite                   ["AT5G65840","AT3G54440","AT3G50560","AT1G29465","AT1G64990","AT1G30270","AT2G35490","AT4G05400","AT5G01775","AT4G32570","AT5G45260","AT3G10270","AT4G25990","AT5G12170","AT1G26120","AT1G22430","AT1G07650","AT3G62750","AT4G39235","AT2G24150","AT5G19460","AT3G18790","AT2G47760","AT2G04630","AT2G30420","AT3G02070","AT3G24320","AT3G58110","AT3G15820","AT2G30740","AT5G66360","AT3G16260", "AT5G07240", "AT1G78260","AT4G15630","AT1G54730"]
# overlap repressed composite extend 3prime     ["AT4G34215","AT2G46710"]
# overlap repressed same exon                   ["AT4G01050","AT4G14160", "AT5G61310", "AT5G21070", "AT1G01510", "AT1G07320", "AT1G19000", "AT1G32200", "AT3G52420", "AT5G58370", "AT1G09130" ,"AT3G45050","AT1G27435", "AT4G16950", "AT1G02720", "AT5G17310", "AT2G36300","AT1G57540", "AT1G33490", "AT3G56800","AT2G30550","AT1G18745", "AT1G72320", "AT1G63855"]

# enhanced skipped                              ["AT2G18300", "AT5G15020", "AT3G63190", "AT4G10320", "AT1G23170", "AT1G68830", "AT1G71720", "AT3G21250", "AT5G10690"]
# enhanced composite                            ["AT5G61220", "AT4G19110","AT1G60260", "AT1G69550", "AT2G31440"]
# enhanced_same_exon                            ["AT5G46450", "AT3G24440", "AT3G20230", "AT3G55010", "AT1G72910", "AT2G16090", "AT4G28860", "AT2G32070", "AT3G18890", "AT1G64385", "AT5G13930", "AT5G57270"] 

# "AT4G38670", "AT3G54900", "AT1G70185", "AT1G15405", "AT3G01500"


for i in gene_ids:
    
    igv = IGV(i)
    igv.add_bam(nanopore_bam_file1, sort_method='3', label = "combined amirU1-70K")
    igv.add_bam(nanopore_bam_file2, sort_method='3', label = "combined amirU1-C")
    igv.add_bam(nanopore_bam_file3, sort_method='3', label = "combined Col-0")
  


    igv.add_bedgraph(bedgraph1) 
    igv.add_bedgraph(bedgraph2)
    igv.add_bedgraph(bedgraph3)
    igv.add_bedgraph(bedgraph4)
    igv.add_bedgraph(bedgraph5)
    igv.add_bedgraph(bedgraph6)
    igv.add_bedgraph(bedgraph7)
    igv.add_bedgraph(bedgraph8)
    igv.add_bedgraph(bedgraph9)
    
    igv.plot(height=2.0, width=5.6, height_bedgraph = 1, extend_3_prime_by = 500)